### Link to homework
https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/01-docker-terraform/homework.md


In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from time import time

In [2]:
trip_data_file_path = "/workspaces/data-engineering-zoomcamp/data/green_tripdata_2019-09.csv.gz"
green_table_name = "green_tripdata_2019_09"

In [27]:
encoded_password = quote_plus("P@ssw0rd!")
print(encoded_password)
engine = create_engine(f"postgresql://postgres:{encoded_password}@db:5432/ny_taxi")

P%40ssw0rd%21


In [28]:
engine.connect()

### Question 1. Knowing docker tags
Run the command to get information on Docker
docker --help

Now run the command to get help on the "docker build" command:
docker build --help

Do the same for "docker run".
Which tag has the following text? - Automatically remove the container when it exits

Answer:
**--rm**

### Question 2. Understanding docker first run
Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash. Now check the python modules that are installed ( use pip list ).

What is version of the package wheel ?

Answer:
root@c3816d7e6e77:/# wheel version

wheel 0.45.1

### Question 4. Longest trip for each day
Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every trip on a single day, we only care about the trip with the longest distance.

2019-09-18
2019-09-16
2019-09-26
2019-09-21


In [29]:
query = f"""
SELECT COUNT(*) FROM {green_table_name}
"""

pd.read_sql_query(query, engine)

,count
0,449063


In [30]:
query = f"""
SELECT * FROM {green_table_name}
ORDER BY trip_distance DESC
LIMIT 1
"""

pd.read_sql_query(query, engine)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-09-26 19:32:52,2019-09-27 01:50:27,N,4.0,265,16,1.0,341.64,1761.0,1.0,0.5,0.0,0.0,None,0.3,1762.8,2.0,1.0,0.0


Answer: the longest trip was on 2019-09-26

### Question 5. Three biggest pick up Boroughs
Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

"Brooklyn" "Manhattan" "Queens";
"Bronx" "Brooklyn" "Manhattan";
"Bronx" "Manhattan" "Queens";
"Brooklyn" "Queens" "Staten Island";

In [41]:
query = f"""
SELECT *
FROM information_schema.columns
WHERE table_name = '{green_table_name}'
LIMIT 5
"""

pd.read_sql(query, engine)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,ny_taxi,public,green_tripdata_2019_09,congestion_surcharge,20,None,YES,double precision,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,ny_taxi,public,green_tripdata_2019_09,lpep_pickup_datetime,2,None,YES,timestamp without time zone,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,ny_taxi,public,green_tripdata_2019_09,lpep_dropoff_datetime,3,None,YES,timestamp without time zone,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,ny_taxi,public,green_tripdata_2019_09,VendorID,1,None,YES,double precision,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,ny_taxi,public,green_tripdata_2019_09,RatecodeID,5,None,YES,double precision,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [50]:
query = f"""
SELECT SUM(total_amount) AS total_amount
FROM {green_table_name} AS G
INNER JOIN taxi_zones AS T ON G."PULocationID" = T."LocationID"
WHERE T.Borough = 'Manhattan' OR T.Borough = 'Brooklyn' OR T.Borough = 'Queens'
"""
# --

result = pd.read_sql(query, engine)

print(result.to_string(float_format='{:,.2f}'.format))

   total_amount
0  7,531,951.89


In [51]:
query = f"""
SELECT SUM(total_amount) AS total_amount
FROM {green_table_name} AS G
INNER JOIN taxi_zones AS T ON G."PULocationID" = T."LocationID"
WHERE T.Borough = 'Bronx' OR T.Borough = 'Brooklyn' OR T.Borough = 'Manhattan'
"""
# --


result = pd.read_sql(query, engine)

print(result.to_string(float_format='{:,.2f}'.format))

   total_amount
0  5,865,417.52


In [52]:
query = f"""
SELECT SUM(total_amount) AS total_amount
FROM {green_table_name} AS G
INNER JOIN taxi_zones AS T ON G."PULocationID" = T."LocationID"
WHERE T.Borough = 'Bronx' OR T.Borough = 'Manhattan' OR T.Borough = 'Queens'
"""
# --"Bronx" "Manhattan" "Queens";


result = pd.read_sql(query, engine)

print(result.to_string(float_format='{:,.2f}'.format))

   total_amount
0  5,730,731.41


In [53]:
query = f"""
SELECT SUM(total_amount) AS total_amount
FROM {green_table_name} AS G
INNER JOIN taxi_zones AS T ON G."PULocationID" = T."LocationID"
WHERE T.Borough = 'Brooklyn' OR T.Borough = 'Queens' OR T.Borough = 'Staten Island'
"""
# --"Brooklyn" "Queens" "Staten Island";


result = pd.read_sql(query, engine)

print(result.to_string(float_format='{:,.2f}'.format))

   total_amount
0  5,115,319.28


### Question 6. Largest tip
For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

Note: it's not a typo, it's tip , not trip

Central Park
Jamaica
JFK Airport
Long Island City/Queens Plaza